In [34]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('../Benchmark/GoogleNews-vectors-negative300.bin', binary=True)


In [139]:
from pyemd import emd 
import pickle
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import gensim, math
import time
import pandas as pd
import nltk 
from os.path import dirname, abspath
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import lda
import numpy as np
import gensim
from nltk import word_tokenize
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
import string
import os
from scipy.spatial.distance import cosine
from gensim.matutils import softcossim
from scipy.sparse import csc_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import mixture
from collections import Counter
import sklearn.metrics as mt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Data sets

In [55]:
from sklearn.datasets import fetch_rcv1
rcv1 = fetch_rcv1()


KeyError: (1,)

# BBC news

In [154]:
class dotdict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

topics = ["business", "entertainment", "politics", "sport", "tech"]
cwd = dirname(os.getcwd())+"/Data/bbc2/"
BBCNews = dotdict({'data': [],
           'label': []})
noise_indices= random.sample(range(0, len(noises)), 2225)
add_noise = True 
counter = 0
nn = 100
for i, s in enumerate(topics):
    path = cwd+s
    for filename in os.listdir(path):
        with open(path+"/"+filename, 'r', encoding="latin-1") as f:
            text = f.read() 
            text = text+(" "+noises[noise_indices[counter]]+" ")*nn if add_noise else text
            BBCNews.data.append(text)
            BBCNews.label.append(i)
        f.close()
        counter+=1


In [ ]:
seeds_indices = random.sample(range(2225), 222)
noises = [BBCNews.data[t] for t in seeds_indices]
noises = " ".join(noises)
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space    
noises = noises.translate(translator).lower().split()
noises = list(set(noises))
noises = [t for t in noises if t in word2vec]

In [141]:
#pickle.dump(texts, open("sample_texts.pickle", "wb"))
texts = pickle.load(open("sample_texts.pickle", "rb"))

In [171]:
x = np.array([[0,1,2,3,4],[2,3,4,5,6]])
x[:,:3]

array([[0, 1, 2],
       [2, 3, 4]])

# Topic soft cosine distance
build_vec() transforms a text into a vector.
Cluster_TSCD() computes the soft cosine similarity between two text vectors.

In [199]:
#similarity_matrix = np.ones((len(allTopics), len(allTopics)), np.float64)
#for i in range(1, len(allTopics)):
#    for j in range(i):
#        x = time.time()
#        distance_matrix[i, j] = euclidean(allTopics[i], allTopics[j])
#        similarity_matrix[i, j] = 1-cosine(allTopics[i], allTopics[j])
#        similarity_matrix[j, i] = similarity_matrix[i, j]

def build_vec_auto(text, cls, strategy):
    """build vector represenetation of text
        cls: Clustering model
        strategy: used with spectral clustering 
            0: non-empy clusters, positive silhoutte scores, weighted average
            1: 
    """
    x = text
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space    
    text = text.translate(translator).lower().split()
    start = time.time()
    text = [t for t in text if t in word2vec and not t in stopwords.words("english")]
    if cls == "kmeans":
        num_clusters = math.ceil(np.sqrt(len(set(text))))
        vecs = [word2vec[w] for w in text]
        km = KMeans(n_clusters = num_clusters, random_state=3425, precompute_distances = True)
        km.fit(vecs)
        cluster_sizes = np.array([len(np.where(km.labels_ == i)[0]) for i in range(num_clusters)])
        weights = cluster_sizes/sum(cluster_sizes)
        return np.concatenate([[num_clusters], km.cluster_centers_.flatten(), weights])
    elif cls == "spectral":
        if strategy == 0:
            word_counter = Counter(text)
            unique_text = list(word_counter.keys())
            word_counts = np.array(list(word_counter.values()))
            if len(unique_text) < 3:
                unique_text = unique_text*2
                word_counts = word_counts*2
            num_clusters = math.ceil(np.sqrt(len(unique_text)))
            vecs = np.array([word2vec[w] for w in unique_text])
            sc = SpectralClustering(num_clusters, affinity='precomputed', random_state=3425,
                                    assign_labels = "discretize", n_init=100)
            similarity_matrix = cosine_similarity(vecs)+1
            sc.fit(similarity_matrix)
            silhoutte_scores = silhouette_samples(2-similarity_matrix, sc.labels_, metric = 'precomputed')
            non_empty_clusters = np.array(list(set(sc.labels_)))
            indices = [np.where(sc.labels_ == i) if i in non_empty_clusters 
                                                               else [] for i in range(num_clusters)]
            sh_means = np.array([np.mean(silhoutte_scores[indices[i]]) for i in non_empty_clusters])
            pos_sh_means_indices = np.where(sh_means > 0)
            sh_means = sh_means[pos_sh_means_indices]
            good_clusters = non_empty_clusters[pos_sh_means_indices]
            sh_weights = np.array(sh_means/sum(sh_means))
            
            sizes = np.array([sum(word_counts[indices[i]]) for i in good_clusters])
            sizes_weights = sizes/sum(sizes)
            p = 0.3
            weights = p*sh_weights+(1-p)*sizes_weights
            #print("{},{},{}".format(sum(sh_weights), sum(sizes_weights), sum(weights)))
            centroids = np.array([np.mean(vecs[indices[i]], axis = 0) for i in good_clusters])
            rep = np.concatenate(([len(good_clusters)], centroids.flatten(), sizes_weights))
            return rep
        elif strategy == 1:
            word_counter = Counter(text)
            unique_text = list(word_counter.keys())
            word_counts = list(word_counter.values())
            if len(unique_text) < 3:
                unique_text = unique_text*2
                word_counts = word_counts*2
            num_clusters = math.ceil(np.sqrt(len(unique_text)))
            vecs = [word2vec[w] for w in unique_text]
            sc = SpectralClustering(num_clusters, affinity='precomputed', random_state=3425,
                                    assign_labels = "discretize", n_init=100)
            similarity_matrix = cosine_similarity(vecs)+1
            sc.fit(similarity_matrix)
            silhoutte_scores = silhouette_samples(2-similarity_matrix, sc.labels_, metric = 'precomputed')
            silhoutte_contribution = np.multiply(word_counts, silhoutte_scores)
            total_contribution = sum(silhoutte_contribution)
            non_empty_clusters = list(set(sc.labels_))
            indices = [np.where(sc.labels_ == i)[0].tolist() if i in non_empty_clusters 
                                                               else [] for i in range(num_clusters)]
            
            weights = np.array([sum(np.take(silhoutte_contribution, indices[i]))/total_contribution
                                                                        for i in non_empty_clusters])
            #print("{},{},{}".format(sum(sh_weights), sum(sizes_weights), sum(final_weights)))
            centroids = np.array([np.mean(np.take(vecs, indices[i], axis = 0), axis = 0) for i in non_empty_clusters])
            rep = np.concatenate([[len(non_empty_clusters)], centroids.flatten(), weights])
            #for i in range(len(weights)):
             #   print(weights[i])
              #  print(np.take(unique_text, indices[non_empty_clusters[i]]))
            #print("-----")
            return rep
        elif strategy == 2:
            word_counter = Counter(text)
            unique_text = list(word_counter.keys())
            word_counts = list(word_counter.values())
            if len(unique_text) < 3:
                unique_text = unique_text*2
                word_counts = word_counts*2
            num_clusters = 10#math.ceil(np.sqrt(len(unique_text)))
            vecs = [word2vec[w] for w in unique_text]
            sc = SpectralClustering(num_clusters, affinity='precomputed', random_state=3425,
                                    assign_labels = "discretize", n_init=100)
            similarity_matrix = cosine_similarity(vecs)+1
            sc.fit(similarity_matrix)
            non_empty_clusters = list(set(sc.labels_))
            indices = [np.where(sc.labels_ == i)[0].tolist() if i in non_empty_clusters 
                                                               else [] for i in range(num_clusters)]
            sizes = np.array([sum(np.take(word_counts, indices[i])) for i in non_empty_clusters])
            centroids = np.array([np.mean(np.take(vecs, indices[i], axis = 0), axis = 0) for i in non_empty_clusters])
            rep = np.concatenate([[len(non_empty_clusters)], centroids.flatten(), [1/len(sizes)]*len(sizes)])
            return rep

def cluster_TSCD_auto(a, b):
    def euclidean(a, b):
        return np.sqrt(np.sum(np.square(a-b)))
    start = time.time()
    num_clusters_a, num_clusters_b = int(a[0]), int(b[0])
    cut_a, cut_b = 1+300*num_clusters_a, 1+300*num_clusters_b
    topics1, topics2 = a[1:cut_a].reshape(-1,300), b[1:cut_b].reshape(-1,300)
    weights1, weights2 = a[cut_a:cut_a+num_clusters_a], b[cut_b:cut_b+num_clusters_b]
    allTopics = np.concatenate([topics1,topics2])
    d1 = np.concatenate([weights1,[0]*len(topics2)])
    d2 = np.concatenate([[0]*len(topics1),weights2])
    similarity_matrix = cosine_similarity(allTopics)
    result = d1.T.dot(similarity_matrix).dot(d2)
    vec1 = d1.T.dot(similarity_matrix).dot(d1)
    vec2 = d2.T.dot(similarity_matrix).dot(d2)
    result /= (np.sqrt(vec1)*np.sqrt(vec2))
    #print(time.time()-start)
    return np.clip(1-result, 0 , 1)

def cluster_TSCD(a, b, num_clusters):
    def euclidean(a, b):
        return np.sqrt(np.sum(np.square(a-b)))
    cut = 300*num_clusters
    topics1, topics2 = a[0:cut].reshape(-1,300), b[0:cut].reshape(-1,300)
    freq1, freq2 = a[cut:], b[cut:]
    allTopics = np.concatenate([topics1,topics2])
    
    d1 = np.concatenate([freq1/sum(freq1),[0]*len(topics2)])
    d2 = np.concatenate([[0]*len(topics1),freq2/sum(freq2)])
    similarity_matrix = np.ones((len(allTopics), len(allTopics)), np.float64)
    for i in range(1, len(allTopics)):
        for j in range(i):
            #distance_matrix[i, j] = euclidean(allTopics[i], allTopics[j])
            similarity_matrix[i, j] = 1-cosine(allTopics[i], allTopics[j])
            similarity_matrix[j, i] = similarity_matrix[i, j]
    result = d1.T.dot(similarity_matrix).dot(d2)
    vec1 = d1.T.dot(similarity_matrix).dot(d1)
    vec2 = d2.T.dot(similarity_matrix).dot(d2)
    result /= (np.sqrt(vec1)*np.sqrt(vec2))
    return np.clip(1-result, 0, 1)

def cluster_TSCD_raw(text1, text2, num_clusters):
    return cluster_TSCD(build_vec(text1, num_clusters), build_vec(text2, num_clusters), num_clusters)

def cluster_TSCD_raw_auto(text1, text2, cls, strategy):
    return cluster_TSCD_auto(build_vec_auto(text1, cls, strategy), build_vec_auto(text2, cls, strategy))

x = build_vec_auto(texts[-1], "spectral", 0)
x

array([ 12.        ,   0.03208542,   0.0886898 , ...,   0.14396887,
         0.05836576,   0.06225681])

# Word average distance


In [118]:
def wad(text):
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    text = text.translate(translator).lower().split()
    #text = [t for t in text if not t in stopwords.words("english")]
    return np.mean([word2vec[w] for w in text if w in word2vec], axis=0)

def wordcount(text):
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    text = text.translate(translator).lower().split()
    text = [t for t in text if t in word2vec]
    return len(list(set(text)))

In [135]:
train_X, test_X, train_y, test_y = train_test_split(BBCNews.data, BBCNews.label, test_size=0.3, random_state=42)
print("started KNN ")
start = time.time()
algo = "wad"
if algo == "wad":
    train_vecs = [ wad(t) for t in train_X]
    test_vecs = [ wad(t) for t in test_X]
elif algo == "LSA":
    LsaVectorizer = make_pipeline(TfidfVectorizer(max_features = 20000, use_idf = True, min_df=2)
                              ,TruncatedSVD(300))
    train_vecs = LsaVectorizer.fit_transform(train_X)
    test_vecs = LsaVectorizer.transform(test_X)
elif algo == "TSCD":
    train_vecs = [ build_vec_auto(t, 'spectral', 2) for t in train_X]
    test_vecs = [ build_vec_auto(t, 'spectral', 2) for t in test_X]
    max_len = np.max(np.concatenate(([len(a) for a in train_vecs], [len(a) for a in test_vecs])))
    train_vecs = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0) for a in train_vecs])
    test_vecs = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0) for a in test_vecs])
print(time.time() - start)
print("Vectorization finished")
#nbrs = GridSearchCV(KNeighborsClassifier(algorithm='ball_tree'), cv = None, 
                    #param_grid={"n_neighbors": range(5,20,5)})
#nbrs = GridSearchCV(KNeighborsClassifier(algorithm='ball_tree', 
 #                           metric=lambda a,b: cluster_TSCD_auto(a, b)), cv = None,
  #                         param_grid={"n_neighbors": range(5,25,5)})


started KNN 
2.479295015335083
Vectorization finished


In [136]:
if algo != "TSCD":
    nbrs = GridSearchCV(KNeighborsClassifier(algorithm='ball_tree'), cv = None, 
                    param_grid={"n_neighbors": range(5,20,5)})
else:
    nbrs = KNeighborsClassifier(algorithm='ball_tree', n_neighbors = 10,
                                metric=lambda a,b: cluster_TSCD_auto(a, b))
nbrs.fit(train_vecs, train_y)
print(time.time() - start)
start = time.time()
#test_vecs = [build_vec(t, num_clusters) for t in news20_test.data]

print("started predicting")
start = time.time()
predicted = nbrs.predict(test_vecs)
print(accuracy_score(predicted, test_y))

10.317711114883423
started predicting
0.940119760479


In [68]:
print(mt.classification_report(predicted, test_y))

             precision    recall  f1-score   support

          0       0.90      0.97      0.93       153
          1       0.93      0.96      0.95       114
          2       0.97      0.85      0.91       136
          3       0.99      0.99      0.99       141
          4       0.94      0.94      0.94       124

avg / total       0.94      0.94      0.94       668

